<a href="https://colab.research.google.com/github/gnoevoy/Inventory_Management_Analysis/blob/main/worksheet_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [ ]:
import pandas as pd
import numpy as np

import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

pio.templates.default = "plotly_dark"

# Dataset

In [ ]:
excel_file = r"..\..\Portfolio Projects\Inventory Management\Datasets\WarmeHands - data.xlsx"

stock = pd.read_excel(excel_file, sheet_name="Stock")
orders = pd.read_excel(excel_file, sheet_name="Orders")
price = pd.read_excel(excel_file, sheet_name="Price")
costs = pd.read_excel(excel_file, sheet_name="Costs")

categories = pd.read_csv(r"..\..\Portfolio Projects\Inventory Management\Datasets\categories.csv")

# Preparing Data

In [ ]:
# stock table

stock["SKU-ID"] = stock["SKU-ID"].astype(str).str.replace(" ", "")
stock["Description"] = stock["Description"].str.title().str.strip()

In [ ]:
# price table

price["ID"] = price["ID"].astype(str).str.replace(" ", "")
price["Retail_Price"] = pd.to_numeric(price["Retail_Price"].astype(str).str.replace("$", ""))

In [ ]:
# categories table

categories_mapping = {"home acce" : "home accessories", "toys" : "toys & edibles"}
categories["category"] = categories["category"].map(categories_mapping).fillna(categories["category"]).astype("category")
categories["ID"] = categories["ID"].str.strip("SKU-")

In [ ]:
# orders table

orders["SKU"] = orders["SKU"].astype(str).str.replace(" ", "")
orders["InvoiceDate"] = pd.to_datetime(orders["InvoiceDate"])
orders["Country"] = orders["Country"].str.split("_", expand=True)[1].str.strip(".")

In [ ]:
# costs table

costs["SKU"] = costs["SKU"].astype(str).str.replace(" ", "")
costs["factory_equipment_rent"] = pd.to_numeric(costs["factory_equipment_rent"].str.replace("..", "."))

costs.drop_duplicates(inplace=True)
costs.reset_index(drop=True, inplace=True)

costs["COGS"] = costs["raw_material"] + costs["factory_labor"] + costs["factory_equipment_rent"]

# Explore Data

In [ ]:
# stock table with prices

stock_merged = (
    pd.merge(stock, price, left_on="SKU-ID", right_on="ID", how="left")
    .merge(costs, left_on="SKU-ID", right_on="SKU", how="left")
    .merge(categories, left_on="SKU-ID", right_on="ID", how="left")
    [["SKU-ID", "category", "Description", "2020_units_sold", "2021_start_stock", "Retail_Price", "COGS"]]
)

source table for the following analysis

In [ ]:
# Revenue and Profit in 2020

stock_merged["Revenue_2020"] = stock_merged["2020_units_sold"] * stock_merged["Retail_Price"]
stock_merged["Profit_2020"] = stock_merged["Revenue_2020"] - (stock_merged["2020_units_sold"] * stock_merged["COGS"])
stock_merged["Pct_revenue_2020"] = (stock_merged["Revenue_2020"] / stock_merged["Revenue_2020"].sum() * 100).round(2)

stock_merged.head()

,SKU-ID,category,Description,2020_units_sold,2021_start_stock,Retail_Price,COGS,Revenue_2020,Profit_2020,Pct_revenue_2020
0,82486,decoration,3 Drawer Antique White Wood Cabinet,440,917,13.32,8.95,5860.80,1922.80,0.98
1,23435,decoration,3 Raffia Ribbons Vintage Christmas,692,1033,1.88,0.83,1300.96,726.60,0.22
2,85034B,decoration,3 White Choc Morris Boxed Candles,1610,1142,2.47,1.26,3976.70,1948.10,0.66
3,84559A,toys & edibles,3D Sheet Of Dog Stickers,918,620,1.90,0.85,1744.20,963.90,0.29
4,23697,office & school,A Pretty Thank You Card,557,530,1.30,0.42,724.10,490.16,0.12


In [ ]:
# "quantity_sold" for each item by year

units_sold_by_year = (
    orders[orders["InvoiceDate"].dt.year == 2021]
    .groupby("SKU")
    .agg(total_quntity = ("Quantity", "sum"))
    .rename(columns={"total_quntity" : "2021_units_sold"})
    .reset_index()
)

units_sold_by_year.head()

,SKU,2021_units_sold
0,10125,1149
1,15030,143
2,16054,1164
3,17014A,130
4,17038,1423


In [ ]:
# join tables: stock + quantity

stock_merged_2 = pd.merge(stock_merged, units_sold_by_year, left_on="SKU-ID", right_on="SKU", how="left").drop(columns=["SKU", "category", "Description", "SKU-ID", "2020_units_sold"])

# revenue 2021 + pct
stock_merged_2["Revenue_2021"] = stock_merged_2["2021_units_sold"] * stock_merged_2["Retail_Price"]
stock_merged_2["Pct_revenue_2021"] = (stock_merged_2["Revenue_2021"] / stock_merged_2["Revenue_2021"].sum() * 100).round(2)

stock_merged_2.head()

,SKU-ID,category,Description,2020_units_sold,2021_start_stock,Retail_Price,COGS,Revenue_2020,Profit_2020,Pct_revenue_2020,2021_units_sold,Revenue_2021,Pct_revenue_2021
0,82486,decoration,3 Drawer Antique White Wood Cabinet,440,917,13.32,8.95,5860.80,1922.80,0.98,742.0,9883.44,2.22
1,23435,decoration,3 Raffia Ribbons Vintage Christmas,692,1033,1.88,0.83,1300.96,726.60,0.22,827.0,1554.76,0.35
2,85034B,decoration,3 White Choc Morris Boxed Candles,1610,1142,2.47,1.26,3976.70,1948.10,0.66,888.0,2193.36,0.49
3,84559A,toys & edibles,3D Sheet Of Dog Stickers,918,620,1.90,0.85,1744.20,963.90,0.29,357.0,678.30,0.15
4,23697,office & school,A Pretty Thank You Card,557,530,1.30,0.42,724.10,490.16,0.12,456.0,592.80,0.13


In [ ]:
# create "turnover" table with calculations
# avg_inventory_value + turnover_rate for 2021

turnover = stock_merged_2.loc[:, ["SKU-ID", "category", "Description", "2021_start_stock", "2021_units_sold", "COGS"]]

# calculated columns "2021_end_stock", "Avg_inventory" and "Inventory_turnover"
turnover["2021_end_stock"] = turnover["2021_start_stock"] - turnover["2021_units_sold"]
turnover["Avg_inventory"] = ((turnover["2021_start_stock"] + turnover["2021_end_stock"]) * turnover["COGS"] / 2).round(2)
turnover["Inventory_turnover"] = ((turnover["2021_units_sold"] * turnover["COGS"]) / turnover["Avg_inventory"]).round(2)

turnover.head()

,SKU-ID,category,Description,2021_start_stock,2021_units_sold,COGS,2021_end_stock,Avg_inventory,Inventory_turnover
0,82486,decoration,3 Drawer Antique White Wood Cabinet,917,742.0,8.95,175.0,4886.70,1.36
1,23435,decoration,3 Raffia Ribbons Vintage Christmas,1033,827.0,0.83,206.0,514.19,1.33
2,85034B,decoration,3 White Choc Morris Boxed Candles,1142,888.0,1.26,254.0,879.48,1.27
3,84559A,toys & edibles,3D Sheet Of Dog Stickers,620,357.0,0.85,263.0,375.28,0.81
4,23697,office & school,A Pretty Thank You Card,530,456.0,0.42,74.0,126.84,1.51


In [ ]:
turnover["Inventory_turnover"].mean()

0.922970297029703

In [ ]:
# turnover rate by months

units_sold_by_months = (
    orders[orders["InvoiceDate"].dt.year == 2021]
    .groupby(["SKU", orders["InvoiceDate"].dt.month])
    .agg(total_quntity = ("Quantity", "sum"))
    .rename(columns={"total_quntity" : "2021_units_sold"})
    .reset_index()
)

units_sold_by_months.head()

,SKU,InvoiceDate,2021_units_sold
0,10125,1,211
1,10125,2,9
2,10125,3,23
3,10125,4,152
4,10125,5,127


In [ ]:
example_1 = pd.merge(stock_merged, units_sold_by_months, left_on="SKU-ID", right_on="SKU", how="left").drop(columns=["SKU", "2020_units_sold", "Revenue_2020", "Profit_2020", "Pct_revenue_2020", "Retail_Price"])
example_1 = example_1.rename(columns={"InvoiceDate" : "Month"}).sort_values(["SKU-ID", "Month"], ascending=True)


# calculated columns "2021_end_stock", "Avg_inventory" and "Inventory_turnover"
example_1["2021_end_stock"] = example_1["2021_start_stock"] - example_1["2021_units_sold"]
example_1["Avg_inventory"] = ((example_1["2021_start_stock"] + example_1["2021_end_stock"]) * example_1["COGS"] / 2).round(2)
example_1["Inventory_turnover"] = ((example_1["2021_units_sold"] * example_1["COGS"]) / example_1["Avg_inventory"]).round(2)

example_1.head(20)

,SKU-ID,category,Description,2021_start_stock,COGS,Month,2021_units_sold,2021_end_stock,Avg_inventory,Inventory_turnover
386,10125,office & school,Mini Tapes,1356,0.84,1.0,211.0,1145.0,1050.42,0.17
387,10125,office & school,Mini Tapes,1356,0.84,2.0,9.0,1347.0,1135.26,0.01
388,10125,office & school,Mini Tapes,1356,0.84,3.0,23.0,1333.0,1129.38,0.02
389,10125,office & school,Mini Tapes,1356,0.84,4.0,152.0,1204.0,1075.20,0.12
390,10125,office & school,Mini Tapes,1356,0.84,5.0,127.0,1229.0,1085.70,0.10
391,10125,office & school,Mini Tapes,1356,0.84,6.0,14.0,1342.0,1133.16,0.01
392,10125,office & school,Mini Tapes,1356,0.84,7.0,179.0,1177.0,1063.86,0.14
393,10125,office & school,Mini Tapes,1356,0.84,8.0,60.0,1296.0,1113.84,0.05
394,10125,office & school,Mini Tapes,1356,0.84,9.0,45.0,1311.0,1120.14,0.03
395,10125,office & school,Mini Tapes,1356,0.84,10.0,221.0,1135.0,1046.22,0.18


In [ ]:
example_1[["SKU-ID", "Month", "Inventory_turnover"]].head(20)

,SKU-ID,Month,Inventory_turnover
386,10125,1.0,0.17
387,10125,2.0,0.01
388,10125,3.0,0.02
389,10125,4.0,0.12
390,10125,5.0,0.10
391,10125,6.0,0.01
392,10125,7.0,0.14
393,10125,8.0,0.05
394,10125,9.0,0.03
395,10125,10.0,0.18


In [ ]:
# ABC analysis

abc_analysis = stock_merged_2.loc[:, ["SKU-ID", "category", "Description", "Revenue_2021", "Pct_revenue_2021"]].sort_values("Pct_revenue_2021", ascending=False)
abc_analysis.dropna(how="any", inplace=True)

# comulative % of revenue
abc_analysis["Cumulative_pct_revenue"] = abc_analysis["Pct_revenue_2021"].cumsum()

# abc categories
abc_analysis["ABC_category"] = np.where(abc_analysis["Cumulative_pct_revenue"] < 70, "A",
    np.where(abc_analysis["Cumulative_pct_revenue"].between(70, 90), "B", "C")
)

# rank column
abc_analysis["Rank"] = abc_analysis["Pct_revenue_2021"].rank(method="first", ascending=False).astype(int)

abc_analysis.head()

,SKU-ID,category,Description,Revenue_2021,Pct_revenue_2021,Cumulative_pct_revenue,ABC_category,Rank
42,22693,home accessories,Grow A Flytrap Or Sunflower In Tin,33717.97,7.56,7.56,A,1
86,22619,toys & edibles,Set Of 6 Soldier Skittles,26176.26,5.87,13.43,A,2
100,20719,home accessories,Woodland Charlotte Bag,24014.10,5.38,18.81,A,3
28,23077,toys & edibles,Doughnut Lip Gloss,22600.55,5.07,23.88,A,4
89,21232,decoration,Strawberry Ceramic Trinket Box,22205.30,4.98,28.86,A,5


In [ ]:
# reserach existing graphs and structure for presentation
# ask chat how could I imporve my analysis
    # additional challenging questions

In [ ]:
categories.head()

,ID,category
0,10125,office & school
1,15030,office & school
2,16054,office & school
3,17038,office & school
4,20719,home accessories


In [ ]:
orders

,InvoiceNo,SKU,Quantity,InvoiceDate,Country
0,539639,20932,1,2021-01-01,Australia
1,539659,22677,20,2021-01-02,Australia
2,540027,84849B,12,2021-01-16,Australia
3,540094,21672,12,2021-01-16,Australia
4,540247,22637,3,2021-01-17,Australia
...,...,...,...,...,...
13328,539437,21672,2,2022-01-29,USA
13329,539453,22260,6,2022-01-29,USA
13330,539453,90114,1,2022-01-29,USA
13331,539453,46776B,2,2022-01-29,USA
